In [ ]:
import osmnx as ox
import geopandas as gpd
import rioxarray as rio
import cartopy.crs as ccrs
from matplotlib.colors import LogNorm
import matplotlib.pyplot as plt
from geocube.api.core import make_geocube
import xarray as xr
import numpy as np
from matplotlib_scalebar import ScaleBar
plt.rcParams['font.family'] = 'DejaVu Sans Mono'

In [ ]:
np_4326 = ox.features.features_from_place(
    'Cairngorms National Park',
    tags={'boundary':'protected_area',
          'designation':'national_park'}
    ).reset_index()

np_4326 = np_4326.loc[np_4326['name']=='Cairngorms National Park',
                      ['element_type','osmid','geometry',
                       'name','source','type',
                       'designation','website']].reset_index()

np_4326.to_file(
    'national_park.geojson',
    driver='GeoJSON'
    )

boundary = np_4326.loc[0, 'geometry']

peaks_4326 = ox.features_from_polygon(
    boundary,
    tags={'natural': 'peak'}
    ).reset_index()

peaks_4326['ele'] = peaks_4326['ele'].astype('float32')

peaks_4326.drop(columns=['alt_name:gd','name:gd'], inplace=True)

peaks_4326.to_file(
    'peaks.geojson',
    driver='GeoJSON'
    )

In [ ]:
##### go and run script in ArcGIS Pro
# import arcpy

# arcpy.conversion.JSONToFeatures(
#     'C:/Users/~/30DMC23/day20_outdoors/peaks.geojson',
#     'peaks',
#     geometry_type='POINT'
# )

# crs = arcpy.SpatialReference(4326)
# arcpy.management.DefineProjection('peaks',coor_system=crs)

# arcpy.agolservices.Viewshed(
#     InputPoints='peaks',
#     MaximumDistance=50,
#     MaximumDistanceUnits='Kilometers',
#     DEMResolution='FINEST',
#     ObserverHeight=1.5,
#     ObserverHeightUnits='Meters'
# )

# arcpy.conversion.PolygonToRaster(
#     in_features="Output Viewshed",
#     value_field="Frequency",
#     out_rasterdataset=r"C:/Users/~/30DMC23/day20_outdoors/viewshed.tif",
#     cell_assignment="CELL_CENTER",
#     priority_field="NONE",
#     cellsize=90,
#     build_rat="BUILD"
# )

In [ ]:
# read in exported viewshed tif
vs = rio.open_rasterio('viewshed.tif')

prj = ccrs.epsg(vs.rio.crs.to_epsg())

peaks = peaks_4326.to_crs(vs.rio.crs)
npark = np_4326.to_crs(vs.rio.crs)

np_prj = (make_geocube(np_4326.to_crs(prj),
                       measurements=['index'],
                       output_crs=prj,
                       resolution=90)
          ['index']
          .rio.reproject_match(vs,
                               nodata=0)
)

vs_clip = xr.where(np_prj != 0,
                   vs,
                   0)

np_prj = (make_geocube(np_4326.to_crs(prj),
                       measurements=['index'],
                       output_crs=prj,
                       resolution=90)
          ['index']
          .rio.reproject_match(vs,
                               nodata=0)
)

vs_clip = xr.where(np_prj != 0,
                   vs,
                   0) 

In [ ]:
fig,ax=plt.subplots(figsize=[10,10],
                    subplot_kw={'projection':prj})

vs_clip.coarsen({'x':5,'y':5},
                boundary='trim'
                ).mean().plot(ax=ax,
                              cmap='magma',
                              norm=LogNorm(vmin=1),
                              add_colorbar=False,
                            #   cbar_kwargs={'label':'viewcount',
                            #                'shrink':0.6, 'pad':0.02,
                            #                'orientation':'horizontal'}
                              )
               
peaks.plot(ax=ax, fc='w', ec='k', marker='^', markersize=15, linewidth=0.8)

npark.plot(ax=ax, fc='none', ec='w', linewidth=1.5)

minx,miny,maxx,maxy = npark.total_bounds

ax.set_xlim(minx-5000,maxx+5000)
ax.set_ylim(miny-5000,maxy+5000)

ax.set_facecolor('k')
ax.set_title('')
ax.set_title('  visibility\n   in the\n    cairngorms', loc='left',y=0.8,color='w')